<a href="https://colab.research.google.com/github/tomasonjo/blogs/blob/master/llm/autogen_neo4j.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install --quiet pyautogen langchain-community neo4j

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.6/284.6 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 14.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 203.0/203.0 kB 11.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 14.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.7/296.7 kB 16.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.7/320.7 kB 17.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 29.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 973.5/973.5 kB 30.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.2/310.2 kB 22.3 MB/s eta 0:00:00
    

In [4]:
import os
from langchain_community.graphs import Neo4jGraph

os.environ["NEO4J_URI"] = 'neo4j+s://demo.neo4jlabs.com'
os.environ["NEO4J_USERNAME"] = 'recommendations'
os.environ["NEO4J_PASSWORD"] = 'recommendations'
os.environ["NEO4J_DATABASE"] = 'recommendations'
os.environ["OPENAI_API_KEY"] = 'sk-'

graph = Neo4jGraph(sanitize=True)

In [6]:
from typing import Dict

def execute_query(cypher_query: str) -> Dict:
    return graph.query(cypher_query)

In [11]:
from autogen import ConversableAgent

# Let's first define the assistant agent that suggests tool calls.
assistant = ConversableAgent(
    name="Assistant",
    system_message="You are a helpful AI assistant. "
    "You can help with answering questions about movies and actors. "
    "The graph schema is " + graph.schema +
    "Return 'TERMINATE' when the task is done.",
    llm_config={"config_list": [{"model": "gpt-4o", "api_key": os.environ["OPENAI_API_KEY"]}]},
)

# The user proxy agent is used for interacting with the assistant agent
# and executes tool calls.
user_proxy = ConversableAgent(
    name="User",
    llm_config=False,
    is_termination_msg=lambda msg: msg.get("content") is not None and "TERMINATE" in msg["content"],
    human_input_mode="NEVER",
)

# Register the tool signature with the assistant agent.
assistant.register_for_llm(name="graph_retrieval", description="Graph query execution")(execute_query)

# Register the tool function with the user proxy agent.
user_proxy.register_for_execution(name="graph_retrieval")(execute_query)

<function __main__.execute_query(cypher_query: str) -> Dict>

In [12]:
chat_result = user_proxy.initiate_chat(assistant, message="What is a good comedy?")

User (to Assistant):

What is a good comedy?

--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...
Assistant (to User):

Sure, I can recommend some good comedy movies. Would you like suggestions based on highly rated comedies, popular comedies, or something specific?

--------------------------------------------------------------------------------
User (to Assistant):



--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...
Assistant (to User):

***** Suggested tool call (call_bb5a5uSAzpIWHFBZyO8GskjX): graph_retrieval *****
Arguments: 
{"cypher_query":"MATCH (m:Movie)-[:IN_GENRE]->(:Genre {name: 'Comedy'}) RETURN m.title AS title ORDER BY m.imdbRating DESC LIMIT 1"}
********************************************************************************

--------------------------------------------------------------------------------

>>>>>>>> EXECUTING FUNCTION graph_retrieval..